In [ ]:
!git clone https://github.com/Kacper-M-Michalik/splatter-image

fatal: destination path 'splatter-image' already exists and is not an empty directory.


In [ ]:
!pip install -r splatter-image/requirements.txt

In [ ]:
!pip install datasets

In [ ]:
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git

Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 1053, done.
remote: Total 1053 (delta 0), reused 0 (delta 0), pack-reused 1053 (from 1)
Receiving objects: 100% (1053/1053), 78.70 MiB | 15.29 MiB/s, done.
Resolving deltas: 100% (604/604), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core.git) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization.git) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim.git) registered for path 'submodules/fused-ssim'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3293, done.        
remote: Counting objects: 100% (322/322), done.        
remote: Compressing objects: 100% (17

In [ ]:
%cd /content/gaussian-splatting
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.0;7.5;8.0+PTX"
!pip install -e submodules/diff-gaussian-rasterization
%cd ..

/content/gaussian-splatting
Obtaining file:///content/gaussian-splatting/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
  Running setup.py develop for diff_gaussian_rasterization
/content


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.9.0+cu126
CUDA Version: 12.6


If there is a need to delete any training progress and start from scratch without re-uploads etc, use the following

In [1]:
# !rm -rf wandb model_latest.pth model_best.pth evaluation_results scores.txt .hydra outputs multirun

Finetuning (training the adapters)

In [ ]:
!WANDB_MODE=disabled python splatter-image/train_network.py \
    +dataset=cars_priors \
    opt.lora_finetune=True \
    opt.pretrained_hf=True \
    data.subset=-1 \
    hydra.run.dir=. \
    logging.ckpt_iterations=100

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

[INFO] L

Running the evaluation script

In [ ]:
!python splatter-image/eval.py cars_priors \
    --experiment_path . \
    --save_vis 5 \
    --out_folder evaluation_results

Evaluating on dataset cars_priors
Loading a local model according to the experiment path
Loaded model!
Started downloading datasets
Downloaded datasets
Converted poses
Converted rgbs
Converted depths
Converted normals
Dataset intrin length: 704
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pt